In [ ]:
import ollama



class Player:
    def __init__(self, name: str):
        self.name = name
        self.character = ""

    def check_character(self, character):
        return (self.character == character)



class LLMPlayer(Player):
    MODEL_NAME="EEVE-Korean-10.8B"
    SYSTEM_PROMPT = ("""
        당신은 양세찬 게임 전문 참가자입니다. 반드시 다음 규칙을 따라야 합니다:

        1. 질문 생성 규칙
        - 예/아니오로 답변 가능한 형태
        - 캐릭터 이름 직접 언급 금지
        - 주제와 관련된 내용

        2. 답변 규칙
        - 반드시 사실에 기반해 답변
        - '예', '아니오'로만 답변
        - 추가 설명 없이 3단어 이내로 응답

        3. 추측 규칙
        - 반드시 한 명의 '인물'로만 답변
        - 불확실하면 '모름'이라고 답변
    """)
    


    def __init__(self, name):
        super().__init__(name)        
        self.messages = [{"role": "system", "content": self.__class__.SYSTEM_PROMPT}]



    # 자신 캐릭터 관련 질문 (Question)
    def get_Q_res(self, content):
        self.messages.append({"role": "user", "content": content})
       
        q_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
        )['message']['content']

        self.messages.append({"role": "assistant", "content": q_res})

        return q_res
    


    # 상대 질문에 대한 답 (Answer)
    def get_A_res(self, content):
        a_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=[
                {"role": "system", "content": self.__class__.SYSTEM_PROMPT},
                {"role": "user", "content": content}
            ],
        )['message']['content']

        return a_res
    


    # 내 캐릭터 유추 (Result)
    def get_R_res(self, content):
        self.messages.append({"role": "user", "content": content})
       
        r_res = ollama.chat(
            model=self.__class__.MODEL_NAME,
            messages=self.messages,
        )['message']['content']

        self.messages.append({"role": "assistant", "content": r_res})

        return r_res



    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

In [ ]:
# 주제 목록
TOPICS = ["연예인", "영화 인물", "드라마 인물", "위인", "캐릭터"]

# 각 주제별 예시 인물
CHARACTERS = {
    "연예인": [
        "유재석", "아이유", "BTS 진", "블랙핑크 제니", "송강호", "김태리", "이영자", "박보검", "전지현", "이병헌",
        "공유", "한효주", "김수현", "김고은", "박명수", "이승기", "박서준", "정해인", "차은우", "수지"
    ],
    "영화 인물": [
        "해리 포터", "헤르미온느 그레인저", "론 위즐리", "인디아나 존스", "제임스 본드", "다스 베이더", "루크 스카이워커", "한 솔로", "조커", "배트맨",
        "포레스트 검프", "토니 스타크", "캡틴 아메리카", "엘사", "올라프", "도로시", "토토", "한니발 렉터", "토니 몬타나", "존 윅"                
    ],
    "드라마 인물": [
        "김신", "지은탁", "이헌", "고애신", "도민준", "천송이", "성덕선", "최택", "박새로이", "조이서",
        "김주원", "길라임", "이강", "채송화", "이준혁", "차수현", "윤세리", "리정혁", "장만월", "구찬성"
    ],
    "위인": [
        "세종대왕", "이순신", "간디", "아인슈타인", "링컨", "나폴레옹", "클레오파트라", "칭기즈칸", "넬슨 만델라", "마리 퀴리",
        "플라톤", "아리스토텔레스", "에디슨", "테슬라", "다윈", "갈릴레이", "코페르니쿠스", "뉴턴", "파스퇴르", "멘델"
    ],
    "캐릭터": [
        "도라에몽", "피카츄", "뽀로로", "미키마우스", "슈퍼마리오", "소닉", "톰", "제리", "스폰지밥", "심슨",
        "짱구", "곰돌이 푸", "미니언", "토토로", "포뇨", "둘리", "고길동", "베지터", "카카로트", "루피"
    ]
}

In [ ]:
import random

def random_topic():    
    return random.choice(TOPICS)


def random_characters(topic, player_count):
    print(CHARACTERS[topic])
    return random.choices(CHARACTERS[topic], k=player_count)

In [ ]:
def create_players(player_count):
    players = {}

    # User
    players["user"] = Player("user")

    # AI
    for i in range(1, player_count):
        name = f"ai-{i}"
        players[name] = LLMPlayer(name)
    
    return players



# 게임 준비
player_count = 3
players = create_players(player_count)

In [ ]:
# 주제 및 캐릭터 할당
topic = random_topic()
characters = random_characters(topic, player_count)

# 캐릭터 할당
for (other_player, character) in zip(players.values(), characters):
    other_player.character = character

In [ ]:
# PROMPT 정의
QUESTION_PROMPT = (
    "다음 조건에 맞는 질문을 생성하세요:\n"
    "1. 내 캐릭터를 추론할 수 있는 내용\n"
    "2. 캐릭터 이름직접 언급 금지\n"
    "3. '예/아니오' 혹은 단답형으로 답변 가능할 수 있는 내용\n\n"

    "예시:\n"
    "- 예.\n"
    "- 네.\n"
    "- 아니오.\n"
    "- 아뇨.\n"
    "- 남자입니까?\n"
    "- 여자입니까?\n"
    "- 한국인입니까?\n"
    "- 외국인입니까?\n"
)

ANSWER_PROMPT = (
    "질문 '{question}'에 대해 캐릭터('{character}')가 맞는지 답변:\n"
    "1. '예'/'아니오' 또는 '단답형'으로만 답변\n"
    "2. 캐릭터 이름('{character}')을 응답에 포함시키지 말 것\n"
    "3. 사실에 기반한 대답\n\n"

    "예시:\n"
    "- 예, 밀짚모자를 씁니다.\n"
    "- 아니오, 미국드라마입니다.\n"
    "- 모름\n"
    "- 네, 맞아요.\n"
    "- 아니요, 아닙니다.\n"
    "- 남자예요.\n"
    "- 여자예요.\n"
    "- 한국인이에요.\n"
    "- 외국인이에요.\n"
)

GUESS_PROMPT = (
    "현재까지의 정보로 '내 캐릭터가 누구인지 추측'하세요:\n"
    "1. 반드시 한 명의 '인물' 또는 '가상인물'로 답변\n"
    "2. 모를 경우 '모름'\n\n"

    "예시:\n"
    "- 유재석\n"
    "- 해리포터\n"
    "- 둘리\n"
    "- 김구\n"
    "- 박효신\n"
    "- 모름\n"
    "- 포레스트 검프\n"
    "- 토니 스타크\n"
    "- 캡틴 아메리카\n"
    "- 길라임\n"
    "- 이강\n"
    "- 채송화\n"
    "- 플라톤\n"
    "- 아리스토텔레스\n"
    "- 에디슨\n"
    "- 테슬라\n"
    "- 짱구\n"
    "- 곰돌이 푸\n"
    "- 미니언\n"
    "- 토토로\n"
)


def play_game(players, topic, max_rounds=20):
    print(f"게임 시작!!!\n>>>> 주제 : '{topic}'\n")
    print("각 플레이어에게 캐릭터를 할당했했습니다.")
    for (name, player) in players.items():
        print(f"- {name} : {player.character}")
    
    winner = None
    round_num = 1

    while round_num <= max_rounds and winner is None:
        print(f"\n--- {round_num}라운드 ---")
        
        for (player_name, player) in players.items():
            player = players[player_name]
            print(f"\n[{player_name}의 차례]")
            
            # 질문
            question=""
            if (player_name == "user"):
                question = input("내 캐릭터에 대한 질문 >> ")
            else:
                question = player.get_Q_res(QUESTION_PROMPT)
            print(f"'{player_name}' 질문 >> {question}")
            
            # 답변
            for (other_name, other_player) in players.items():
                # 질문한 당사자는 건너뜀
                if (other_name == player_name):
                    continue
                
                answer=""
                if (other_name == "user"):
                    answer = input(f"{player_name}의 질문에 답변 >> ")
                else:
                    answer = other_player.get_A_res(
                        ANSWER_PROMPT.format(
                            character=player.character,
                            question=question
                        ),
                    )
                print(f"{other_name} 답변 >> {answer}")

            
            # 정답
            guess=""
            if (player_name == "user"):
                guess = input("정답 (패스 Enter) : ")
            else:
                guess = player.get_R_res(GUESS_PROMPT)
            print(f"{player_name} 정답 >> {guess}")


            # 정답 판별
            if guess and guess.strip() == player.character:
                print(f"🎉 {player_name}님이 정답을 맞췄습니다! 게임 종료")
                winner = player_name
                break
            else:
                print(f"{player_name}님은 정답을 맞추지 못했습니다.")

        round_num += 1

    if winner:
        print(f"\n게임 승자: {winner} (정답: {players[winner].character})")
    else:
        print("\n아쉽게도 아무도 정답을 맞히지 못했습니다.")


# 게임 플레이 시작
play_game(players, topic)